## IAS CoRoT lightcurves spatialized Sonification

Data download: http://idoc-corot.ias.u-psud.fr/sitools/client-user/COROT_N2_PUBLIC_DATA/project-index.html

The CoRoT space mission, launched on 2006 December 27, was developed and operated by the CNES, with participation of the Science Programs of ESA, ESA's RSSD, Austria, Belgium, Brazil, Germany and Spain.


In [ ]:
from astropy.io import fits
from astropy.timeseries import BoxLeastSquares
import matplotlib.pylab as plt
import numpy as np

import os
from pathlib import Path

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.visualization.wcsaxes.frame import EllipticalFrame
from astropy.coordinates import Angle, SkyCoord
from mocpy import MOC, World2ScreenMPL
import astropy.units as u

In [ ]:
import pygame.midi
import librosa
import time
from pythonosc import udp_client
from math import log

### Loading data

In [ ]:
flux_data = np.load('flux_data.npy', allow_pickle=True)
jds_data = np.load('jds_data.npy', allow_pickle=True)
mk_list = np.load('mk_list.npy')
lumclass_list = np.load('lumclass_list.npy')
mean_list = np.load('mean_list.npy')
rms_list = np.load('rms_list.npy')
period_list = np.load('period_list.npy')
ra_list = np.load('ra_list.npy')
dec_list = np.load('dec_list.npy')
names_list = np.load('names_list.npy')
dimensions_list = np.load('dimensions_list.npy')

### Fitting and cleaning data

In [ ]:
min_mean = min(mean_list)
max_mean = max(mean_list)
print(min_mean, max_mean)

In [ ]:
Y_min, Y_max = 0.2, 1 
mean = Y_min + (Y_max - Y_min) * (mean_list - min_mean) / (max_mean - min_mean)

In [ ]:
print(min(mean_list), max(mean_list)) # Amplitudes

### Spectral type to note conversion

In [ ]:
mk_list # note / octave

In [ ]:
mk_note = np.copy(mk_list)

In [ ]:
mk_note = np.char.replace(mk_note, "O", "C")

In [ ]:
mk_note = np.char.replace(mk_note, "K", "D")

In [ ]:
mk_note = np.char.replace(mk_note, "M", "E")
mk_note

### Luminosity class conversion to integers

In [ ]:
lumclass_list[0]

In [ ]:
lumclass_list

In [ ]:
ones = 0
twos = 0
threes = 0
fours = 0
fives = 0
sixs = 0
sevens = 0
eights = 0
empties = 0
lumclass_num = np.copy(lumclass_list)
for i in range(len(lumclass_list)):
    if lumclass_list[i] == "I":
        lumclass_num[i] = "1"
        ones += 1
    if lumclass_list[i] == "II":
        lumclass_num[i] = "2"
        twos +=1
    if lumclass_list[i] == "III":
        lumclass_num[i] = "3"
        threes += 1
    if lumclass_list[i] == "IV":
        lumclass_num[i] = "4"
        fours += 1
    if lumclass_list[i] == "V":
        lumclass_num[i] = "5"
        fives += 1
    if lumclass_list[i] == "VI":
        lumclass_num[i] = "6"
        sixs += 1
    if lumclass_list[i] == "VII":
        lumclass_num[i] = "7"
        sevens += 1
    if lumclass_list[i] == "VIII":
        lumclass_num[i] = "8"
        eights += 1
    if lumclass_list[i] == "":
        lumclass_num[i] = "0"
        empties += 1
 

In [ ]:
lumclass_num

In [ ]:
print(len(lumclass_list))
print(ones, twos, threes, fours, fives, sixs, sevens, eights, empties)

### Launching sonification

In [ ]:
pygame.midi.init()
pygame.midi.get_count() #Bluetooth Keyboard #4 => 10 I/O

In [ ]:
#pygame initialization
pygame.midi.init() #The port changes with MIDI I/O bus assignment. Ex: With 4 buses--> 0-3 Input / 4-7 Output
player0 = pygame.midi.Output(5)

curves = 0
previous_note = 0

for i in range(len(mk_list)):
    if mk_list[i] == '':
        i += 1
    else:
        if mk_list[i] == previous_note:
            i += 1
        else:
            try:
                note_on = pygame.midi.frequency_to_midi(librosa.note_to_hz(mk_list[i]))
                
                print("____________________________________")
                print("Loading data: light curve ", curves+1)
                print(names_list[i])
                previous_note = mk_list[i]
               #------------------------------        
                fig, ax = plt.subplots()
                fig.set_size_inches(20., 8.)
                ax.plot(jds_data[i], flux_data[i], 'r.')
              #  ax.set_ylim((mean_list[curves] - 10000), (mean_list[curves] + 10000))
                fig.suptitle(names_list[i])
                ax.set_ylabel("White Flux (e-/32s)")
                ax.set_xlabel("Time (JD)")
                plt.savefig('Lc.png', transparent=True)
                plt.show()

                #------------------------------

                coords = SkyCoord(ra_list , dec_list,frame='icrs',unit='deg')    
                fig2 = plt.figure(figsize=(16,8))

                with World2ScreenMPL(
                    fig2,
                    fov=320 * u.deg,
                    center=SkyCoord(100, 0, unit='deg', frame='icrs'),
                    coordsys="icrs",
                    rotation=Angle(0, u.degree),
                ) as wcs:
                    ax2 = fig2.add_subplot(111, projection=wcs, frame_class=EllipticalFrame)
                    ax2.set_title("CoRoT library")
                    ax2.grid(color="black", linestyle="dotted")
                    ax2.scatter(coords.ra,coords.dec,marker='o', s=10,transform=ax2.get_transform('world'), zorder=10)
                    ax2.scatter(ra_list[i],dec_list[i],marker='*',color = 'r', s=100,transform=ax2.get_transform('world'),zorder=10)

                    plt.savefig('Coord.png', transparent=True)

        #---PLAY----  

                client_s = udp_client.SimpleUDPClient("127.0.0.1", 9989)
                client_s.send_message("/s", 1)

                player0.note_on(note_on, 25)#int(lumclass_list[i])*50)
                client0 = udp_client.SimpleUDPClient("127.0.0.1", 9996)   
                client0.send_message("/ra", ra_list[i])

                client1 = udp_client.SimpleUDPClient("127.0.0.1", 9997)   
                client1.send_message("/dec", dec_list[i])
      #------------------          
                client_2 = udp_client.SimpleUDPClient("127.0.0.1", 9995)
                client_2.send_message("/period", round(period_list[i], 3))

                client_3 = udp_client.SimpleUDPClient("127.0.0.1", 9994)
                client_3.send_message("/mean", round(log(mean_list[i])/10, 2))

                client_4 = udp_client.SimpleUDPClient("127.0.0.1", 9993)
                client_4.send_message("/type", mk_list[i])

                client_5 = udp_client.SimpleUDPClient("127.0.0.1", 9992)
                client_5.send_message("/class", lumclass_list[i])

                client_6 = udp_client.SimpleUDPClient("127.0.0.1", 9991)
                client_6.send_message("/note", mk_note[i])

                client_7 = udp_client.SimpleUDPClient("127.0.0.1", 9990)
                client_7.send_message("/class", int(lumclass_num[i]))


      #------------------
                print("MIDI NOTE: ", note_on)
                print("--------Lightcurve ", curves+1, "--------")
                print("Spectral type: ", mk_list[i])
                print("Luminosity class: ", lumclass_list[i])
                print("LC RMS: ", rms_list[i])
                print("LC Mean: ", mean_list[i])
                print("RA: ", ra_list[i])
                print("DEC: ", dec_list[i])
                print("Bes fit period: ", round(period_list[i], 3), "days")
                curves += 1
                time.sleep(2)
                client_s.send_message("/s", 0)
                player0.note_off(note_on, 0)
                time.sleep(.05)
                curves += 1
            except:
                print("--------Lightcurve ", curves+1, "--------")
                print("ERROR")
                print("MIDI NOTE: ", note_on)
                print("--------Lightcurve ", curves+1, "--------")
                print("Spectral type: ", mk_list[i])
                print("Luminosity class: ", lumclass_list[i])
                print("LC RMS: ", rms_list[i])
                print("LC Mean: ", mean_list[i])
                print("RA: ", ra_list[i])
                print("DEC: ", dec_list[i])
                print("Bes fit period: ", round(period_list[i], 3), "days")
                curves += 1
                #note_on = 200


print ("Light curves loaded: ",curves+1);
del player0    
pygame.midi.quit()




In [ ]:
from math import log
print(log(min(mean_list))/10)
print(log(max(mean_list))/10)
log(mean_list[90])/10